<a href="https://colab.research.google.com/github/abhi9716/notebooks/blob/master/speed_up_QnA_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import logging
logger = logging.getLogger(__name__)

In [5]:
# !pip install onnxruntime
# !pip install --upgrade git+https://github.com/huggingface/transformers
!pip install transformers onnxruntime onnx psutil 
!pip install onnxruntime-tools

     |████████████████████████████████| 2.1MB 3.8MB/s 
     |████████████████████████████████| 4.1MB 38.7MB/s 
     |████████████████████████████████| 12.2MB 51.0MB/s 
     |████████████████████████████████| 901kB 51.7MB/s 
     |████████████████████████████████| 3.3MB 46.9MB/s 
     |████████████████████████████████| 215kB 3.7MB/s 
     |████████████████████████████████| 61kB 3.5MB/s 
     |████████████████████████████████| 51kB 2.9MB/s 
     |████████████████████████████████| 102kB 3.0MB/s 
     |████████████████████████████████| 92kB 3.8MB/s 
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-cp37-none-any.whl size=22245 sha256=1608e1a26fb5def0e1a780d5b653ce552c58914a6bc156b32215f21282252c10
  Stored in directory: /root/.cache/pip/wheels/2e/15/f5/aa2a056d223903b52cf4870134e3a01df0c723816835dd08db
Successfully built py-cpuinfo


In [2]:
# !pip install transformers

In [3]:
# !pip install huggingface_hub
# !huggingface-cli login

In [8]:
%%time
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering
import torch
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased')
question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"
inputs = tokenizer(question, text, return_tensors='pt')
start_positions = torch.tensor([1])
end_positions = torch.tensor([3])
outputs = model(**inputs, start_positions=start_positions, end_positions=end_positions)
loss = outputs.loss
start_scores = outputs.start_logits
end_scores = outputs.end_logits
answer_start = torch.argmax(
        start_scores
    )  # Get the most likely beginning of answer with the argmax of the score
answer_end = torch.argmax(end_scores) + 1  # Get the most likely end of answer with the argmax of the score
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs.input_ids[answer_start:answer_end]))
print(f"Question: {question}")
print(f"Answer: {answer}")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this mode

Question: Who was Jim Henson?
Answer: 
CPU times: user 2 s, sys: 116 ms, total: 2.11 s
Wall time: 2.65 s


In [9]:
%%time
import json
import requests
API_TOKEN="BFOdJbLtxgBzhwYJaDuKJAVcnQmDwuMalejmAtedlXSZINgUBwkInjUoRqZhwjbTVPfUTMLYsCrhtoFsJnsluVjvxUlsvfKnqXrZbITHCNSxHDpWKQftiLjVPbIsjJoL"
headers = {"Authorization": f"Bearer {API_TOKEN}"}
API_URL = "https://api-inference.huggingface.co/models/mrm8488/bert-medium-finetuned-squadv2"

def query(payload):
    data = json.dumps(payload)
    response = requests.request("POST", API_URL, headers=headers, data=data)
    return json.loads(response.content.decode("utf-8"))

data = query(
    {
        "inputs": {
            "question": "when her nine of the marriage occured?",
            "context": """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.""",
        }
    }
)
print(data)

{'score': 0.9679889678955078, 'start': 1286, 'end': 1307, 'answer': 'between 1999 and 2002'}
CPU times: user 15 ms, sys: 843 µs, total: 15.8 ms
Wall time: 294 ms


In [11]:
%%time
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
tokenizer = AutoTokenizer.from_pretrained("mrm8488/bert-medium-finetuned-squadv2")
model = AutoModelForQuestionAnswering.from_pretrained("mrm8488/bert-medium-finetuned-squadv2")

# %%time
text = r""" New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""

print(len(text.split()))
questions = [
    "when her nine of the marriage occured?",
    # "What does 🤗 Transformers provide?",
    # "🤗 Transformers provides interoperability between which frameworks?",
]
for question in questions:
    inputs = tokenizer(question, text, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]
    outputs = model(**inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits
    answer_start = torch.argmax(
        answer_start_scores
    )  # Get the most likely beginning of answer with the argmax of the score
    answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of answer with the argmax of the score
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
    print(f"Question: {question}")
    print(f"Answer: {answer}")

369
Question: when her nine of the marriage occured?
Answer: between 1999 and 2002
CPU times: user 2.07 s, sys: 119 ms, total: 2.19 s
Wall time: 2.81 s


##Tranformers to ONNX

In [13]:
from time import time
from onnxruntime import GraphOptimizationLevel, InferenceSession, SessionOptions, get_all_providers

def create_model_for_provider(model_path: str, provider: str) -> InferenceSession: 
  
  assert provider in get_all_providers(), f"provider {provider} not found, {get_all_providers()}"

  # Few properties that might have an impact on performances (provided by MS)
  options = SessionOptions()
  options.intra_op_num_threads = 1
  options.graph_optimization_level = GraphOptimizationLevel.ORT_ENABLE_ALL

  # Load the model as a graph and prepare the CPU backend 
  session = InferenceSession(model_path, options, providers=[provider])
  session.disable_fallback()
    
  return session


In [14]:
tokenizer.save_pretrained("qa/")
model.save_pretrained("qa/")
from pathlib import Path
path = Path("onnx/qa.onnx")
!rm -rf onnx/
from transformers.convert_graph_to_onnx import convert

# Handles all the above steps for you
convert(framework="pt",  ## pt for pytorch
        model="qa",     ## model path
        output=path, 
        opset=11,
        pipeline_name = "question-answering") ## pipeline_name is most important when you use this function
cpu_model = create_model_for_provider("onnx/qa.onnx", "CPUExecutionProvider")
# %%time
# start_scores, end_scores = cpu_model.run(None, inputs_onnx)


ONNX opset version set to: 11
Loading pipeline (model: qa, tokenizer: qa)
Creating folder onnx
Using framework PyTorch: 1.8.1+cu101
Found input input_ids with shape: {0: 'batch', 1: 'sequence'}
Found input token_type_ids with shape: {0: 'batch', 1: 'sequence'}
Found input attention_mask with shape: {0: 'batch', 1: 'sequence'}
Found output output_0 with shape: {0: 'batch', 1: 'sequence'}
Found output output_1 with shape: {0: 'batch', 1: 'sequence'}
Ensuring inputs are in correct order
position_ids is not present in the generated input list.
Generated inputs order: ['input_ids', 'attention_mask', 'token_type_ids']


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1790: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  input_tensor.shape[chunk_dim] == tensor_shape for input_tensor in input_tensors


In [15]:
%%time
from transformers import AutoTokenizer
from transformers import BertTokenizerFast
import torch


text = r""" New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""

print(len(text.split()))
questions = [
    "when her nine of the marriage occured?",
    # "What does 🤗 Transformers provide?",
    # "🤗 Transformers provides interoperability between which frameworks?",
]
# tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")
tokenizer = AutoTokenizer.from_pretrained("mrm8488/bert-medium-finetuned-squadv2")
inputs = tokenizer(question, text, add_special_tokens=True, return_tensors="pt")

inputs_onnx= {
    'input_ids' : inputs.input_ids.numpy(),
    'attention_mask' : inputs.attention_mask.numpy(), 
    'token_type_ids' : inputs.token_type_ids.numpy(),
}

cpu_model = create_model_for_provider("onnx/qa.onnx", "CPUExecutionProvider")
output=cpu_model.run(None,inputs_onnx)
start_scores = outputs.start_logits
end_scores = outputs.end_logits
# all_tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
# print(tokenizer.convert_tokens_to_string(all_tokens[torch.argmax(torch.tensor(start_scores)) : torch.argmax(torch.tensor(end_scores))+1]))
answer_start = torch.argmax(
        start_scores
    )  # Get the most likely beginning of answer with the argmax of the score
answer_end = torch.argmax(end_scores) + 1  # Get the most likely end of answer with the argmax of the score
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
print(f"Question: {questions[0]}")
print(f"Answer: {answer}")

369
Question: when her nine of the marriage occured?
Answer: between 1999 and 2002
CPU times: user 1.02 s, sys: 538 ms, total: 1.56 s
Wall time: 2.15 s


##optimizations 

https://medium.com/analytics-vidhya/speedup-pytorch-inference-with-onnx-284d6a47936e

In [16]:
from onnxruntime_tools import optimizer
from onnxruntime_tools.transformers.onnx_model_bert import BertOptimizationOptions

# disable embedding layer norm optimization for better model size reduction
opt_options = BertOptimizationOptions('bert')
opt_options.enable_embed_layer_norm = False

opt_model = optimizer.optimize_model(
    "onnx/qa.onnx",
    'bert', 
    num_heads=12,
    hidden_size=768,
    optimization_options=opt_options)
opt_model.save_model_to_file('onnx/qa.opt.onnx')

--num_heads is {self.num_heads}. Detected value is {num_heads}. Using detected value.
--hidden_size is {self.hidden_size}. Detected value is {hidden_size}. Using detected value.
--num_heads is {self.num_heads}. Detected value is {num_heads}. Using detected value.
--hidden_size is {self.hidden_size}. Detected value is {hidden_size}. Using detected value.
--num_heads is {self.num_heads}. Detected value is {num_heads}. Using detected value.
--hidden_size is {self.hidden_size}. Detected value is {hidden_size}. Using detected value.
--num_heads is {self.num_heads}. Detected value is {num_heads}. Using detected value.
--hidden_size is {self.hidden_size}. Detected value is {hidden_size}. Using detected value.
--num_heads is {self.num_heads}. Detected value is {num_heads}. Using detected value.
--hidden_size is {self.hidden_size}. Detected value is {hidden_size}. Using detected value.
--num_heads is {self.num_heads}. Detected value is {num_heads}. Using detected value.
--hidden_size is {self.h

In [18]:
import os
def print_size_of_model(model):
    print('Size (MB):', os.path.getsize(model)/(1024*1024))
    # os.remove('temp.p')
print_size_of_model("/content/onnx/qa.onnx")
print_size_of_model("/content/qa/pytorch_model.bin")

Size (MB): 156.87521266937256
Size (MB): 156.88487911224365


In [19]:
from os import environ
from psutil import cpu_count

# Constants from the performance optimization available in onnxruntime
# It needs to be done before importing onnxruntime
environ["OMP_NUM_THREADS"] = str(cpu_count(logical=True))
print(str(cpu_count(logical=True)))
environ["OMP_WAIT_POLICY"] = 'ACTIVE'

2


## Quantize model
https://github.com/microsoft/onnxruntime/blob/master/onnxruntime/python/tools/quantization/notebooks/bert/Bert-GLUE_OnnxRuntime_quantization.ipynb

In [20]:
def quantize_onnx_model(onnx_model_path, quantized_model_path):
    from onnxruntime.quantization import quantize_dynamic, QuantType
    import onnx
    onnx_opt_model = onnx.load(onnx_model_path)
    quantize_dynamic(onnx_model_path,
                     quantized_model_path,
                     weight_type=QuantType.QInt8)

    logger.info(f"quantized model saved to:{quantized_model_path}")

In [21]:
from transformers.convert_graph_to_onnx import quantize
from onnxruntime.quantization import quantize_dynamic, QuantType

# Transformers allow you to easily convert float32 model to quantized int8 with ONNX Runtime
# quantized_model_path = quantize(Path("/content/onnx/qa.opt.onnx"))
quantized_model_path = quantize_onnx_model(Path("/content/onnx/qa.opt.onnx"),"onnx/qa.opt.quant.onnx")
print('ONNX full precision model size (MB):', os.path.getsize("onnx/qa.opt.onnx")/(1024*1024))
print('ONNX quantized model size (MB):', os.path.getsize("onnx/qa.opt.quant.onnx")/(1024*1024))

ONNX full precision model size (MB): 156.8445987701416
ONNX quantized model size (MB): 39.39502429962158


In [22]:
%%time
from transformers import AutoTokenizer
from transformers import BertTokenizerFast
import torch


text = r""" New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""

print(len(text.split()))
questions = [
    "when her nine of the marriage occured?",
    # "What does 🤗 Transformers provide?",
    # "🤗 Transformers provides interoperability between which frameworks?",
]
# tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")
tokenizer = AutoTokenizer.from_pretrained("mrm8488/bert-medium-finetuned-squadv2")
inputs = tokenizer(question, text, add_special_tokens=True, return_tensors="pt")

inputs_onnx= {
    'input_ids' : inputs.input_ids.numpy(),
    'attention_mask' : inputs.attention_mask.numpy(), 
    'token_type_ids' : inputs.token_type_ids.numpy(),
}

cpu_model = create_model_for_provider("/content/onnx/qa.opt.onnx", "CPUExecutionProvider")
output=cpu_model.run(None,inputs_onnx)
start_scores = outputs.start_logits
end_scores = outputs.end_logits
# all_tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
# print(tokenizer.convert_tokens_to_string(all_tokens[torch.argmax(torch.tensor(start_scores)) : torch.argmax(torch.tensor(end_scores))+1]))
answer_start = torch.argmax(
        start_scores
    )  # Get the most likely beginning of answer with the argmax of the score
answer_end = torch.argmax(end_scores) + 1  # Get the most likely end of answer with the argmax of the score
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
print(f"Question: {questions[0]}")
print(f"Answer: {answer}")

369
Question: when her nine of the marriage occured?
Answer: between 1999 and 2002
CPU times: user 810 ms, sys: 63.3 ms, total: 873 ms
Wall time: 1.38 s


In [23]:
%%time
from transformers import AutoTokenizer
from transformers import BertTokenizerFast
import torch


text = r""" New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""

print(len(text.split()))
questions = [
    "when her nine of the marriage occured?",
    # "What does 🤗 Transformers provide?",
    # "🤗 Transformers provides interoperability between which frameworks?",
]
# tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")
tokenizer = AutoTokenizer.from_pretrained("mrm8488/bert-medium-finetuned-squadv2")
inputs = tokenizer(question, text, add_special_tokens=True, return_tensors="pt")

inputs_onnx= {
    'input_ids' : inputs.input_ids.numpy(),
    'attention_mask' : inputs.attention_mask.numpy(), 
    'token_type_ids' : inputs.token_type_ids.numpy(),
}

cpu_model = create_model_for_provider("/content/onnx/qa.opt.quant.onnx", "CPUExecutionProvider")
output=cpu_model.run(None,inputs_onnx)
start_scores = outputs.start_logits
end_scores = outputs.end_logits
# all_tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
# print(tokenizer.convert_tokens_to_string(all_tokens[torch.argmax(torch.tensor(start_scores)) : torch.argmax(torch.tensor(end_scores))+1]))
answer_start = torch.argmax(
        start_scores
    )  # Get the most likely beginning of answer with the argmax of the score
answer_end = torch.argmax(end_scores) + 1  # Get the most likely end of answer with the argmax of the score
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
print(f"Question: {questions[0]}")
print(f"Answer: {answer}")

369
Question: when her nine of the marriage occured?
Answer: between 1999 and 2002
CPU times: user 596 ms, sys: 16.9 ms, total: 613 ms
Wall time: 983 ms
